In [ ]:
from pathlib import Path
import datetime
import csv

import numpy as np
#import pandas as pd
import polars as pl

import stock
from stock.kabutan.financial import FinancialStatement

In [ ]:
csv_dir = stock.PROJECT_ROOT / "data" /"financial"
csv_list = sorted(csv_dir.glob("*.csv"))

In [ ]:
def calc_mean_average(array: np.ndarray, days: int):
    """
    """
    mean_average = np.convolve(array, np.ones(days), mode="valid") / days
    return mean_average

def check_growing(
        q_data: list[FinancialStatement], 
        key: str, 
        min_growth_rate: float, 
        min_duration: int, 
        current_date: datetime.datetime = datetime.datetime.today(),
        num_average: int = 1
        ):
    """
    Args:
        q_data (List[FinancialStatement]) : quarterly data
    """
    assert num_average > 0, "num_average must be greater than 0."
    target_data = sorted([d for d in q_data if d.announce_date is not None and d.announce_date < current_date])[::-1]
    if len(target_data) < min_duration + 1 + num_average - 1:
        return False
    
    def check_is_last_year_data(cur_data: FinancialStatement, prev_data: FinancialStatement):
        is_last_year = prev_data.year == cur_data.year - 1
        is_last_year &= prev_data.month == cur_data.month
        is_last_year &= prev_data.duration == cur_data.duration
        is_last_year &= prev_data.is_prediction == False
        return is_last_year
    
    is_growing = True
    for i in range(min_duration):
        cur_data = target_data[i:i + num_average]
        prev_data = [next((d for d in target_data if check_is_last_year_data(data, d)), None) for data in cur_data]
        if any([d is None for d in prev_data]):
            is_growing = False
            break

        if any([d.__dict__[key] is None for d in cur_data])  or any([d.__dict__[key] is None for d in prev_data]):
            is_growing = False
            break
        
        cur_val = sum([d.__dict__[key] for d in cur_data])
        prev_val = sum([d.__dict__[key] for d in prev_data])
        if prev_val > -1e-5:
            is_growing &= cur_val / (prev_val + 1e-5) > (1.0 + min_growth_rate)
        else:
            is_growing &= cur_val / (prev_val - 1e-5) < 0
        if not is_growing:
            break
    return is_growing

In [ ]:
growing_list = []
for csv_path in csv_list:
    data = stock.kabutan.read_financial_csv(csv_path)
    quarterly_data = [d for d in data if d.duration == 3]
    yearly_data = [d for d in data if d.duration == 12]

    if check_growing(quarterly_data, "eps", 0.2, 2) or check_growing(quarterly_data, "eps", 0.2, 2, num_average=2):
        growing_list.append(csv_path)

In [ ]:
# def check_trend_template(code: str, current_date: datetime.datetime = datetime.datetime.today()):
code = "1301"
current_date = datetime.datetime.today()

daily_csv_path = stock.PROJECT_ROOT / "data" / "daily" / f"{code}.csv"
financial_csv_path = stock.PROJECT_ROOT / "data" / "financial" / f"{code}.csv"

df = pl.read_csv(daily_csv_path)
df = df.sort("date")

days = 10
close_values = df.get_column("close").to_numpy()
avg200 = calc_mean_average(close_values[-200 - days:], days=200)
avg150 = calc_mean_average(close_values[-150 - days:], days=150)
avg50 = calc_mean_average(close_values[-50 - days:], days=50)

prev_date = current_date - datetime.timedelta(days=365)
datetime_series = df.get_column("date").str.to_datetime("%Y/%m/%d")
wk52_data = df.filter(prev_date < datetime_series & datetime_series <= current_date)
wk52_close = wk52_data.close.to_numpy()

flag = True
# 株価が150日移動平均線と200日移動平均線を上回っている
flag &= np.all(close_values[-days:] > avg200) and np.all(close_values[-days:] > avg150)
# 150日移動平均線が200日移動平均線を上回っている
flag &= np.all(avg150 > avg200)
# 200日移動平均線が上向き
flag &= np.all(avg200[1:] - avg200[:-1] > 0) 
# 52週安値から少なくとも30%高い
flag &= wk52_close.min() * 1.3 < wk52_close[-1]
# 52週高値から少なくとも25%以内にある
flag &= wk52_close.max() * 0.75 < wk52_close[-1]

# return flag

In [ ]:
len(growing_list)

In [ ]:
check_growing(quarterly_data, "eps", -10.0, 16, num_average=3)